<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=de5f04e8a123d917e4e232407c587322a6b05c48b3a9e4b799ee6fc817210656
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-02 15:09:01
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 1.50 L (1.08%)
Current PnL: -21.19 L (-14.07%)
CY Booked + Current PnL: -9.75 L (-6.47%)
-------------------
Total profit:  1.31 L
Total loss:  -22.50 L
-------------------
Total Booked + Current PnL: 17.23 L (13.93%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.51 L (63.04%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113606.0,13102.0,16007.0,0.22,13.04,14.09,28.96,79.0,0.82,0.81,51.43,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.81,37.0,M-SC,2.45,83512.0,-17265.0,17354.0,0.40,-17.13,20.78,0.09,245.0,-0.99,0.60,9.72,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.87,45.0,H-MC,3.07,187910.0,23240.0,20088.0,-0.10,14.11,10.69,26.31,89.0,1.16,1.34,23.22,X40N,NTT,AC
42,ITC,452.00,-38.77,47.0,H-LC,2.04,197996.0,-2142.0,23027.0,0.83,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.70,213720.0,3354.0,25433.0,0.44,1.59,11.90,13.68,37.0,0.13,1.53,18.38,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,75.44,80.0,H-SC,1.96,161160.0,-17911.0,83642.0,18.90,-10.00,51.90,36.71,135.0,-0.21,1.15,59.90,XR,NTT,BANKS
26,GREENPANEL,537.00,221.06,54.0,M-SC,1.25,143448.0,-39630.0,118603.0,6.50,-21.65,82.68,43.14,230.0,-0.33,1.03,34.93,XY24,NTT,MISC
73,TATAMOTORS,1065.00,-17.55,60.0,M-LC,2.29,260634.0,-15889.0,125964.0,5.56,-5.75,48.33,39.81,54.0,-0.13,1.86,24.91,XY24,NTT,AUTO
71,TANLA,1963.11,-21.56,52.0,H-SC,17.93,210758.0,-60003.0,389966.0,4.87,-22.16,185.03,121.86,133.0,-0.15,1.51,64.87,AR,ATH,IT
61,SAMMAANCAP,326.00,-187.50,81.0,M-SC,8.36,151695.0,2475.0,141698.0,4.78,1.66,93.41,96.62,208.0,0.02,1.08,64.17,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.73,24.0,M-SC,1.21,88350.0,-3148.0,81900.0,-4.43,-3.44,92.70,86.07,223.0,-0.04,0.63,28.30,XR,NTT,DURABLES
3,ALKYLAMINE,4546.37,-12.24,28.0,H-SC,10.68,81700.0,-19263.0,113792.0,-3.56,-19.08,139.28,93.63,148.0,-0.17,0.58,21.69,SR,ATH,CHEMICALS
55,RAJOOENG,143.10,-45.47,20.0,H-SC,14.25,82750.0,-19750.0,60350.0,-3.24,-19.27,72.93,39.61,114.0,-0.33,0.59,0.00,AR,ATH,MISC
81,VAIBHAVGBL,521.00,47.66,44.0,H-SC,5.80,128028.0,-54747.0,165822.0,-2.36,-29.95,129.52,60.77,125.0,-0.33,0.92,14.67,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,102.57,31.0,M-SC,1.51,132891.0,-41668.0,41701.0,-1.73,-23.87,31.38,0.02,221.0,-1.00,0.95,15.59,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1242.90,69.0,M-SC,16.61,182709.0,1566.0,43905.0,-0.33,0.86,24.03,25.10,235.0,0.04,1.31,46.42,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-21.57,45.0,H-MC,2.20,134922.0,-866.0,43620.0,1.96,-0.64,32.33,31.48,107.0,-0.02,0.96,11.40,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,42.0,M-LC,5.76,149916.0,-1029.0,110488.0,1.02,-0.68,73.70,72.51,53.0,-0.01,1.07,5.30,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,65.0,H-SC,11.47,130167.0,-12312.0,31839.0,4.01,-8.64,24.46,13.70,163.0,-0.39,0.93,53.63,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,57.0,H-SC,1.59,225000.0,-44667.0,81000.0,1.39,-16.56,36.00,13.47,138.0,-0.55,1.61,13.80,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.81,37.0,M-SC,2.45,83512.0,-17265.0,17354.0,0.40,-17.13,20.78,0.09,245.0,-0.99,0.60,9.72,OX40N,NTT,DURABLES
66,SIS,528.0,2107.25,45.0,H-SC,4.05,89072.0,-21960.0,45569.0,1.19,-19.78,51.16,21.26,156.0,-0.48,0.64,20.34,OX40N,NTT,MISC
18,CERA,9475.0,-23.15,35.0,H-SC,1.51,140312.0,-35591.0,77621.0,-1.01,-20.23,55.32,23.89,149.0,-0.46,1.00,21.18,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1242.90,69.0,M-SC,16.61,182709.0,1566.0,43905.0,-0.33,0.86,24.03,25.10,235.0,0.04,1.31,46.42,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.53,232987.0,8025.0,159619.0,0.61,3.57,68.51,74.53,192.0,0.05,1.67,36.97,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,3.86,209561.0,4485.0,108092.0,-0.15,2.19,51.58,54.89,52.0,0.04,1.50,26.81,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.87,45.0,H-MC,3.07,187910.0,23240.0,20088.0,-0.10,14.11,10.69,26.31,89.0,1.16,1.34,23.22,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,42.0,H-MC,2.84,202740.0,10998.0,26762.0,0.85,5.74,13.20,19.69,99.0,0.41,1.45,12.92,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113606.0,13102.0,16007.0,0.22,13.04,14.09,28.96,79.0,0.82,0.81,51.43,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,9.44,191481.0,11673.0,100738.0,0.88,6.49,52.61,62.52,88.0,0.12,1.37,42.79,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,3.86,209561.0,4485.0,108092.0,-0.15,2.19,51.58,54.89,52.0,0.04,1.50,26.81,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,33.0,H-SC,10.02,127253.0,3917.0,129849.0,1.44,3.18,102.04,108.45,119.0,0.03,0.91,26.24,AR,ATH,MISC
86,WIPRO,420.00,-14.70,42.0,M-LC,5.76,149916.0,-1029.0,110488.0,1.02,-0.68,73.70,72.51,53.0,-0.01,1.07,5.30,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.42,28.0,H-LC,5.83,205894.0,-45874.0,99076.0,-0.44,-18.22,48.12,21.13,27.0,-0.46,1.47,10.75,X40,ATH,PAINTS
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282634.0,-33208.0,145867.0,-0.16,-10.51,51.61,35.67,5.0,-0.23,2.02,1.88,X40N,ATH,FMCG
74,TCS,4476.75,-31.26,33.0,H-LC,12.39,274019.0,-71937.0,146792.0,0.92,-20.79,53.57,21.64,1.0,-0.49,1.96,0.92,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,35.0,H-LC,10.24,218387.0,-23537.0,86809.0,0.43,-9.73,39.75,26.15,8.0,-0.27,1.56,3.20,X40,ATH,IT
44,JIOFIN,387.00,-4.80,39.0,H-LC,12.90,258688.0,-8677.0,74140.0,2.59,-3.25,28.66,24.48,48.0,-0.12,1.85,49.94,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,43.0,H-LC,1.75,231052.0,-15099.0,94315.0,-0.45,-6.13,40.82,32.18,16.0,-0.16,1.65,27.70,X200,ATH,IT
65,SIEMENS,4671.5,-2.61,44.0,H-LC,1.97,156395.0,-29700.0,77181.0,-0.04,-15.96,49.35,25.51,15.0,-0.38,1.12,15.25,AR,ATH,ELECTRICAL
42,ITC,452.0,-38.77,47.0,H-LC,2.04,197996.0,-2142.0,23027.0,0.83,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.79,42.0,H-LC,3.70,213720.0,3354.0,25433.0,0.44,1.59,11.90,13.68,37.0,0.13,1.53,18.38,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.28,46.0,H-LC,3.72,273078.0,7652.0,49127.0,1.22,2.88,17.99,21.40,11.0,0.16,1.95,9.72,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,33.0,H-LC,12.39,274019.0,-71937.0,146792.0,0.92,-20.79,53.57,21.64,1.0,-0.49,1.96,0.92,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282634.0,-33208.0,145867.0,-0.16,-10.51,51.61,35.67,5.0,-0.23,2.02,1.88,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,35.0,H-LC,10.24,218387.0,-23537.0,86809.0,0.43,-9.73,39.75,26.15,8.0,-0.27,1.56,3.20,X40,ATH,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.04,197996.0,-2142.0,23027.0,0.83,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308403.0,-4485.0,176160.0,0.42,-1.43,57.12,54.87,3.0,-0.03,2.21,5.12,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,1.97,156395.0,-29700.0,77181.0,-0.04,-15.96,49.35,25.51,15.0,-0.38,1.12,15.25,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.04,197996.0,-2142.0,23027.0,0.83,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.42,28.0,H-LC,5.83,205894.0,-45874.0,99076.0,-0.44,-18.22,48.12,21.13,27.0,-0.46,1.47,10.75,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.52,42.0,H-LC,3.96,207089.0,2589.0,38829.0,1.59,1.27,18.75,20.25,10.0,0.07,1.48,10.11,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.70,213720.0,3354.0,25433.0,0.44,1.59,11.90,13.68,37.0,0.13,1.53,18.38,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,33.0,H-LC,12.39,274019.0,-71937.0,146792.0,0.92,-20.79,53.57,21.64,1.0,-0.49,1.96,0.92,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308403.0,-4485.0,176160.0,0.42,-1.43,57.12,54.87,3.0,-0.03,2.21,5.12,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.04,197996.0,-2142.0,23027.0,0.83,-1.07,11.63,10.44,4.0,-0.09,1.42,4.43,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282634.0,-33208.0,145867.0,-0.16,-10.51,51.61,35.67,5.0,-0.23,2.02,1.88,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,46.0,H-LC,7.69,249432.0,-12187.0,131401.0,0.25,-4.66,52.68,45.57,7.0,-0.09,1.78,6.14,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,38.26,80064.0,-33485.0,73539.0,1.16,-29.49,91.85,35.27,268.0,-0.46,0.57,95.96,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,42.0,H-SC,12.39,91800.0,-6180.0,27779.0,1.11,-6.31,30.26,22.05,152.0,-0.22,0.66,33.44,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113606.0,13102.0,16007.0,0.22,13.04,14.09,28.96,79.0,0.82,0.81,51.43,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,65.0,H-SC,11.47,130167.0,-12312.0,31839.0,4.01,-8.64,24.46,13.70,163.0,-0.39,0.93,53.63,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-187.50,81.0,M-SC,8.36,151695.0,2475.0,141698.0,4.78,1.66,93.41,96.62,208.0,0.02,1.08,64.17,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1242.90,69.0,M-SC,16.61,182709.0,1566.0,43905.0,-0.33,0.86,24.03,25.10,235.0,0.04,1.31,46.42,XY24,NTT,HEALTHCARE
69,SURYODAY,240.0,75.44,80.0,H-SC,1.96,161160.0,-17911.0,83642.0,18.90,-10.00,51.90,36.71,135.0,-0.21,1.15,59.90,XR,NTT,BANKS
61,SAMMAANCAP,326.0,-187.50,81.0,M-SC,8.36,151695.0,2475.0,141698.0,4.78,1.66,93.41,96.62,208.0,0.02,1.08,64.17,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3613.51,50.0,M-SC,7.49,166770.0,-30362.0,120191.0,2.64,-15.40,72.07,45.57,236.0,-0.25,1.19,18.24,XY24,NTT,AUTO
78,UJJIVANSFB,60.0,106.53,65.0,H-SC,11.47,130167.0,-12312.0,31839.0,4.01,-8.64,24.46,13.70,163.0,-0.39,0.93,53.63,OX40N,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.02
1,25,43.13
2,50,74.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.22
LC    32.06
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      14.47
X40N     11.22
XR       11.12
XY25     10.44
AR        8.10
OX40N     7.77
X200      1.65
MH        1.62
X5K       1.50
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.04
H-LC    25.47
H-MC    20.43
M-SC    13.75
M-LC     5.56
M-MC     2.98
L-SC     1.43
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.83,-6.17,41.21
IT,12.49,-20.44,86.11
FINANCE,10.40,-15.91,68.56
MISC,6.95,-18.43,84.27
BANKS,6.45,-10.66,68.88
PAINTS,5.56,-18.74,36.44
ELECTRICAL,5.33,-10.46,49.22
INSURANCE,3.75,-5.52,44.97
AUTO,3.57,-14.96,69.92


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3250149.0,22
XR,1354696.0,14
AR,1198791.0,9
X40,826345.0,10
X40N,635976.0,9
OX40N,560356.0,10
XY25,475560.0,7
SR,259297.0,2
X5K,108092.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733546.0,29
M-SC,1507878.0,17
H-LC,1257855.0,15
H-MC,1251103.0,15
M-LC,372067.0,4
M-MC,367221.0,2
L-SC,260778.0,3
L-MC,60249.0,1
L-LC,39999.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1231356.0      6
M-SC       XY24       850532.0      7
H-SC       AR         821460.0      5
           XR         780790.0      7
H-MC       XY24       573413.0      4
H-LC       X40        531864.0      5
M-MC       XY24       367221.0      2
H-SC       X40N       313868.0      4
           SR         259297.0      2
           OX40N      255663.0      4
H-LC       X40N       235685.0      3
H-MC       X40        218950.0      4
H-LC       AR         208582.0      2
H-MC       XY25       180594.0      2
L-SC       XR         172215.0      2
M-SC       AR         168749.0      2
M-LC       XY24       153487.0      2
M-SC       XY25       141698.0      1
           OX40N      141152.0      4
           XR         130216.0      2
H-LC       XY25       113269.0      3
M-LC       XR         110488.0      1
           X5K        108092.0      1
H-MC       XR         100738.0      1
H-LC       X200        94315.0      1
L-SC       OX40N       88563.0      1
H-MC       X40N        86423.0      2
M-SC       X40         75531.0      1
H-MC       OX40N       74978.0      1
H-LC       XY24        74140.0      1
H-SC       MH          71112.0      1
L-MC       XR          60249.0      1
L-LC       XY25        39999.0      1
H-MC       MH          16007.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
